# 1_AppliedNLP_Session2_Bi_Trigrams

This notebook analyzes the most frequent **bigrams** and **trigrams** in *War and Peace* and *Anna Karenina* by Leo Tolstoy. The structure follows the same logic as `01_frequent_words(1).ipynb`.

In [4]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import pandas as pd
import string
import re

# Ensure tokenizer resources are available (handles newer NLTK too)
def _ensure_nltk():
    try:
        nltk.data.find("tokenizers/punkt")
    except LookupError:
        nltk.download("punkt", quiet=True)
    # Some NLTK versions require 'punkt_tab' separately
    try:
        nltk.data.find("tokenizers/punkt_tab")
    except LookupError:
        try:
            nltk.download("punkt_tab", quiet=True)
        except Exception:
            pass

_ensure_nltk()


### 🔍 What does `Counter` do?
`Counter` from Python's `collections` module counts how many times each item appears in a list. For example:
```python
Counter(['apple', 'banana', 'apple'])
```
returns:
```
Counter({'apple': 2, 'banana': 1})
```
This helps us find how frequently each bigram or trigram occurs.

In [6]:
import os

# Construct the path to the data folder (one level up from notebooks/)
data_dir = os.path.join(os.path.dirname(os.getcwd()), "data")

file_war = os.path.join(data_dir, "The Project Gutenberg eBook of War and Peace, by Leo Tolstoy.txt")
file_anna = os.path.join(data_dir, "The Project Gutenberg eBook of Anna Karenina, by Leo Tolstoy.txt")


def load_and_clean_text(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.lower()
    for p in string.punctuation:
        text = text.replace(p, ' ')
    # Tokenize with NLTK; if resources are missing, fall back to regex
    try:
        tokens = word_tokenize(text)
    except LookupError:
        # Fallback: simple regex tokenizer to avoid NLTK data errors
        tokens = re.findall(r"[a-zA-Z]+", text)
    tokens = [t for t in tokens if t.isalpha()]  # keep only alphabetic tokens
    return tokens

tokens_war = load_and_clean_text(file_war)
tokens_anna = load_and_clean_text(file_anna)

print(f"Loaded {len(tokens_war)} tokens from War and Peace")
print(f"Loaded {len(tokens_anna)} tokens from Anna Karenina")


Loaded 572173 tokens from War and Peace
Loaded 359574 tokens from Anna Karenina


In [7]:

# Generate bigrams and trigrams
bigrams_war = list(ngrams(tokens_war, 2))
bigrams_anna = list(ngrams(tokens_anna, 2))
trigrams_war = list(ngrams(tokens_war, 3))
trigrams_anna = list(ngrams(tokens_anna, 3))

# Count frequencies
counter_bi_war = Counter(bigrams_war)
counter_bi_anna = Counter(bigrams_anna)
counter_tri_war = Counter(trigrams_war)
counter_tri_anna = Counter(trigrams_anna)
    

In [11]:

# Convert to DataFrame for easy display
def top_ngrams(counter, n=20):
    df = pd.DataFrame(counter.most_common(n), columns=['N-gram', 'Frequency'])
    df['N-gram'] = df['N-gram'].apply(lambda x: ' '.join(x))
    df.index = df.index + 1  # Shift index to start at 1
    return df


print("Top 20 Bigrams - War and Peace")
display(top_ngrams(counter_bi_war))

print("Top 20 Bigrams - Anna Karenina")
display(top_ngrams(counter_bi_anna))

print("Top 20 Trigrams - War and Peace")
display(top_ngrams(counter_tri_war))

print("Top 20 Trigrams - Anna Karenina")
display(top_ngrams(counter_tri_anna))
    

Top 20 Bigrams - War and Peace


,N-gram,Frequency
1,of the,4072
2,in the,2336
3,to the,2329
4,and the,1482
5,at the,1346
6,on the,1334
7,he had,1210
8,prince andrew,1065
9,did not,1048
10,he was,951


Top 20 Bigrams - Anna Karenina


,N-gram,Frequency
1,of the,1945
2,in the,1634
3,to the,1007
4,he had,1004
5,he was,886
6,at the,886
7,and the,729
8,it was,673
9,on the,655
10,did not,643


Top 20 Trigrams - War and Peace


,N-gram,Frequency
1,he did not,223
2,i don t,203
3,one of the,187
4,out of the,178
5,that he was,155
6,commander in chief,147
7,as soon as,146
8,up to the,129
9,he could not,129
10,that it was,125


Top 20 Trigrams - Anna Karenina


,N-gram,Frequency
1,i don t,254
2,he could not,198
3,he did not,197
4,out of the,177
5,i can t,142
6,that he had,136
7,said stepan arkadyevitch,125
8,that he was,121
9,in spite of,116
10,that it was,107


In [12]:

# Merge bigrams comparison
def merge_comparison(counter1, counter2, label1, label2, n=20):
    all_ngrams = set(counter1) | set(counter2)
    data = []
    for ng in all_ngrams:
        data.append({
            'N-gram': ' '.join(ng),
            f'{label1} Count': counter1.get(ng, 0),
            f'{label2} Count': counter2.get(ng, 0)
        })
    df = pd.DataFrame(data)
    df['Total'] = df[f'{label1} Count'] + df[f'{label2} Count']
    df = df.sort_values(by='Total', ascending=False).head(n)
    return df

print("### Merged Comparison (Bigrams)")
display(merge_comparison(counter_bi_war, counter_bi_anna, "War and Peace", "Anna Karenina"))

print("### Merged Comparison (Trigrams)")
display(merge_comparison(counter_tri_war, counter_tri_anna, "War and Peace", "Anna Karenina"))
    

### Merged Comparison (Bigrams)


,N-gram,War and Peace Count,Anna Karenina Count,Total
15249,of the,4072,1945,6017
97900,in the,2336,1634,3970
240749,to the,2329,1007,3336
149160,at the,1346,886,2232
31844,he had,1210,1004,2214
83530,and the,1482,729,2211
139661,on the,1334,655,1989
98841,he was,951,886,1837
123702,did not,1048,643,1691
48061,it was,881,673,1554


### Merged Comparison (Trigrams)


,N-gram,War and Peace Count,Anna Karenina Count,Total
276443,i don t,203,254,457
357813,he did not,223,197,420
301850,out of the,178,177,355
43749,he could not,129,198,327
98068,that he was,155,121,276
164143,one of the,187,81,268
348322,that he had,102,136,238
610752,that it was,125,107,232
95821,i can t,80,142,222
151796,as soon as,146,61,207


In [13]:

# Display both separately again for clarity
print("### Separate Comparison (Bigrams)")
print("\nTop 10 shared bigrams:")
shared_bi = set(counter_bi_war) & set(counter_bi_anna)
shared_bi_counts = [(ng, counter_bi_war[ng] + counter_bi_anna[ng]) for ng in shared_bi]
shared_bi_sorted = sorted(shared_bi_counts, key=lambda x: x[1], reverse=True)[:10]
for ng, count in shared_bi_sorted:
    print(f"{' '.join(ng)} — Total: {count}")

print("\n### Separate Comparison (Trigrams)")
shared_tri = set(counter_tri_war) & set(counter_tri_anna)
shared_tri_counts = [(ng, counter_tri_war[ng] + counter_tri_anna[ng]) for ng in shared_tri]
shared_tri_sorted = sorted(shared_tri_counts, key=lambda x: x[1], reverse=True)[:10]
for ng, count in shared_tri_sorted:
    print(f"{' '.join(ng)} — Total: {count}")
    

### Separate Comparison (Bigrams)

Top 10 shared bigrams:
of the — Total: 6017
in the — Total: 3970
to the — Total: 3336
at the — Total: 2232
he had — Total: 2214
and the — Total: 2211
on the — Total: 1989
he was — Total: 1837
did not — Total: 1691
it was — Total: 1554

### Separate Comparison (Trigrams)
i don t — Total: 457
he did not — Total: 420
out of the — Total: 355
he could not — Total: 327
that he was — Total: 276
one of the — Total: 268
that he had — Total: 238
that it was — Total: 232
i can t — Total: 222
as soon as — Total: 207



### 📊 Summary of Comparison
Both books share many common language patterns typical of 19th-century literature.  
* Common bigrams like **'of the'**, **'in the'**, or **'to the'** appear frequently in both.  
* Unique n-grams often reflect thematic differences — *War and Peace* includes more military or political terms, while *Anna Karenina* contains more social and emotional expressions.  
You can analyze further by filtering stopwords or increasing the `n` value for deeper contextual insights.
